In [2]:
import get_news
import pandas as pd
from pprint import pprint

In [3]:
new_path = './news_20191220.pkl'
#df = get_news.incrental_load('./news_20191220.pkl')

# Incrementally loading data
df = pd.read_pickle(new_path)

In [4]:
df.head()

,timestamp,headline,link,content
0,"Last updated December 20, 2019 19:38:56 AEDT",‘ABSOLUTELY PETRIFIED’: Drug dealer sobs as he...,https://www.news.com.au/national/courts-law/te...,A terrified drug dealer has sobbed and called ...
1,"Last updated December 20, 2019 19:38:57 AEDT",'Do you even care?' Firey blasts ScoMo,https://www.news.com.au/technology/environment...,A fire station officer has posted an emotional...
2,"Last updated December 20, 2019 19:38:58 AEDT",Roads melting in extreme heat,https://www.news.com.au/technology/environment...,Roads in parts of South Australia are “bleedin...
3,"Last updated December 20, 2019 19:38:58 AEDT",‘Grossly immoral’: Christians lash Trump,https://www.news.com.au/finance/work/leaders/u...,A major Christian magazine has turned against ...
4,"Last updated December 20, 2019 19:38:58 AEDT",‘This is ridiculous’: Teen’s freakish act,https://www.news.com.au/sport/cricket/big-bash...,The Hobart Hurricanes looked like they were st...


## Stem/Tokenise

Tokenise is basically a function converts sentses into "tokens" or a list of words. In function below we did **stem** on top of the tokenised words, and removed **stop words** from the tokens

In [5]:
#%%
import nltk
import re
from nltk.tokenize import punkt
from nltk.stem import SnowballStemmer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.stem import WordNetLemmatizer 
  
#sample = df.sample(1).content.iloc[0]
keep = re.compile('[a-zA-Z]')

ps = PorterStemmer()
lemmatizer = WordNetLemmatizer() 

def stem_tokenise(corpus):    
    return [lemmatizer.lemmatize(ps.stem(w.lower())) for w in word_tokenize(corpus) if re.match(keep, w)]

# stem tokenise the stop words as well
stop_words = stem_tokenise(" ".join(list(set(stopwords.words('english')))))

D:\python\webscrapping\.env\lib\site-packages\nltk\decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
D:\python\webscrapping\.env\lib\site-packages\nltk\lm\counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence, defaultdict
D:\python\webscrapping\.env\lib\site-packages\nltk\lm\vocabulary.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Counter, Iterable


In [6]:
import spacy

def cleansing(text):    
    # Remove Emails
    text = re.sub('\S*@\S*\s?', '', text) 

    # Remove new line characters
    text = re.sub('\s+', ' ', text)

    # Remove distracting single quotes
    text = re.sub("\'", "", text)
    
    return text

data = df.content.map(cleansing).map(lambda x: " ". join(stem_tokenise(x)))


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
#nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
#data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
#print(data_lemmatized[:2])



<>:5: DeprecationWarning: invalid escape sequence \S
<>:8: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \S
<>:8: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \S
<>:8: DeprecationWarning: invalid escape sequence \s
<ipython-input-6-739772a47b3b>:5: DeprecationWarning: invalid escape sequence \S
  text = re.sub('\S*@\S*\s?', '', text)
<ipython-input-6-739772a47b3b>:8: DeprecationWarning: invalid escape sequence \s
  text = re.sub('\s+', ' ', text)


In [7]:
print(data.to_list()[300])

one night a woman heard blind rattl in the room next door hour later her neighbour s bodi wa found in a pool of blood warn graphic how mani australian die by homicid each year and the horrifi way they are lose their live peter dupa arriv at the suprem court.sourc news limit warn graphic thi octob it final look like polic would be abl to have their day in court over the shock nurs home murder of kathleen down have charg one of australia s most deprav serial killer mr down had live at the brunswick lodg nurs home on quiet loyola ave for eight year becom known a the matriarch of the commun despit her age she wa still sprite a person who wa abl to get up and get about a she wa later describ geoff down son of kathleen down who wa stab three time in the neck the onli suspect is peter dupa and the famili is call for an inquest.sourc news limit a is custom a member of staff check on mr down at on new year s eve and found her soundli sleep at anoth resid heard blind rattl and a door open no dou

## SKlearn DTM

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

Below is an example of sklearn LDA, the problem is it is pretty slow and not as customisable compared to gensim.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# use own tokeniser and stopwords (note: do the same processing for stop words)
#cv = CountVectorizer(stop_words=stop_words, tokenizer=stem_tokenise)

cv = CountVectorizer(analyzer='word',       
         min_df=10,                        # minimum reqd occurences of a word 
         stop_words='english',             # remove stop words
         lowercase=True,                   # convert all words to lowercase
         token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
         # max_features=50000,             # max number of uniq words
)


# get the dtm (coverting the matrix into dataframe)
data_cv = cv.fit_transform(data)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = df.index
#pprint(data_dtm)

# calculate the sparsity
data_dense = data_cv.todense()
# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  10.002603858444786 %


In [9]:
import numpy as np
row = data_dtm.iloc[300]
#pprint(row.nlargest(10))
#pprint(data.iloc[300])

## SKlearn Latent Direchlet Allocation

https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html

In [10]:
# simple LDA
lda = LatentDirichletAllocation(n_components=10, max_iter = 10, learning_method = 'online', batch_size = 128, random_state=0)
lda_output = lda.fit_transform(data_cv)

In [11]:
print("log likelihood: ", lda.score(data_cv))
print("Perplexity: ", lda.perplexity(data_cv))
pprint(lda.get_params())

log likelihood:  -764181.6115431432
Perplexity:  861.1281883103927
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 10,
 'n_jobs': None,
 'perp_tol': 0.1,
 'random_state': 0,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


### Perform a grid search

In [ ]:
from sklearn.model_selection import GridSearchCV

search_params = {'n_components': [10, 20, 30], 'learning_decay': [0.5, 0.7, 0.9]}
lda = LatentDirichletAllocation()
model = GridSearchCV(lda, param_grid=search_params)
model.fit(data_cv)

#best model
best_model = model.best_estimator_

In [ ]:
print("best model parameters: ", best_model.get_params())
result = pd.DataFrame(model.cv_results_)
result

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

plt.figure(figsize=(12, 8))
for x in result.param_learning_decay.unique():
    plt.plot(result[result.param_learning_decay==x].param_n_components, result[result.param_learning_decay==x].mean_test_score, label=x)

plt.title("Choosing Optimal LDA Model")
plt.xlabel("Num Topics")
plt.ylabel("Log Likelyhood Scores")
plt.legend(title='Learning decay', loc='best')
plt.show()

## Finding Dominate Topic

In [ ]:
# Create Document - Topic Matrix
lda_output = best_model.transform(data_cv)

# column names
topicnames = ["Topic" + str(i) for i in range(best_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(df))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(20).style.applymap(color_green).applymap(make_bold)
df_document_topics

## Review Topics Distribution

In [ ]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

## Visualize the LDA model with pyLDAvis

In [ ]:
import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_model, data_cv, cv, mds='tsne')

pyLDAvis.display(panel)

In [ ]:
topics = np.argmax(best_model.transform(data_cv), axis=1)

In [ ]:
topics